In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [1]:
import joblib
from sklearn.ensemble import RandomForestClassifier

# Create a scikit-learn model
model = RandomForestClassifier()

# Train the model on your data

# Save the model to a file
joblib.dump(model, 'EWE.pkl')

['EWE.pkl']

## 1. Data Generation (simulating climate data related to extreme weather events)


In [2]:
def generate_extreme_weather_data(n_samples=1000):
    np.random.seed(42)
    
    # Features: global_temp_anomaly, ocean_temp, co2_levels, el_nino_index
    global_temp_anomaly = np.random.normal(1, 0.5, n_samples)
    ocean_temp = np.random.normal(20, 3, n_samples)
    co2_levels = np.random.normal(410, 20, n_samples)
    el_nino_index = np.random.normal(0, 1, n_samples)
    
    # Target: extreme_weather_probability (simplified model)
    extreme_weather_probability = (
        0.3 * global_temp_anomaly +
        0.2 * (ocean_temp - 20) / 3 +
        0.3 * (co2_levels - 400) / 20 +
        0.2 * el_nino_index +
        np.random.normal(0, 0.1, n_samples)
    )
    extreme_weather_probability = 1 / (1 + np.exp(-extreme_weather_probability))  # Sigmoid to get probability
    
    data = pd.DataFrame({
        'global_temp_anomaly': global_temp_anomaly,
        'ocean_temp': ocean_temp,
        'co2_levels': co2_levels,
        'el_nino_index': el_nino_index,
        'extreme_weather_probability': extreme_weather_probability
    })
    
    return data

## 2. Data Preprocessing


In [3]:
def preprocess_data(data):
    X = data[['global_temp_anomaly', 'ocean_temp', 'co2_levels', 'el_nino_index']]
    y = data['extreme_weather_probability']
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    return X_train, X_test, y_train, y_test, scaler

## 3. Model Creation


In [4]:
def create_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=input_shape),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

## 4. Model Training


In [5]:
def train_model(model, X_train, y_train):
    history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=0)
    return history

## 5. Question Generation


In [6]:
def generate_question(model, scaler, question_type='multiple_choice'):
    # Generate random input
    input_data = np.random.rand(1, 4)
    scaled_input = scaler.transform(input_data)
    
    # Predict extreme weather probability
    predicted_prob = model.predict(scaled_input)[0][0]
    
    # Create question based on the prediction
    if question_type == 'multiple_choice':
        question = "Given the following climate conditions:\n"
        question += f"Global Temperature Anomaly: {input_data[0][0]:.2f}°C\n"
        question += f"Ocean Temperature: {input_data[0][1]:.2f}°C\n"
        question += f"CO2 Levels: {input_data[0][2]:.2f} ppm\n"
        question += f"El Niño Index: {input_data[0][3]:.2f}\n\n"
        question += "What is the most likely scenario for extreme weather events?\n"
        
        options = [
            f"A) High probability of extreme weather (>{predicted_prob+0.2:.2f})",
            f"B) Moderate probability of extreme weather ({predicted_prob-0.1:.2f} - {predicted_prob+0.1:.2f})",
            f"C) Low probability of extreme weather (<{predicted_prob-0.2:.2f})",
            "D) No significant change in extreme weather patterns"
        ]
        
        correct_answer = 'B'
        
        return question, options, correct_answer
    
    elif question_type == 'true_false':
        statement = "Given the current climate conditions, "
        if predicted_prob > 0.5:
            statement += "there is a high likelihood of extreme weather events occurring."
            correct_answer = True
        else:
            statement += "the likelihood of extreme weather events is relatively low."
            correct_answer = False
        
        return statement, correct_answer

## 6. Main Execution


In [7]:
if __name__ == "__main__":
    # Generate and preprocess data
    data = generate_extreme_weather_data()
    X_train, X_test, y_train, y_test, scaler = preprocess_data(data)
    
    # Create and train model
    model = create_model((X_train.shape[1],))
    history = train_model(model, X_train, y_train)
    
    # Evaluate model
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    # Generate questions
    print("\nMultiple Choice Question:")
    mc_question, mc_options, mc_answer = generate_question(model, scaler, 'multiple_choice')
    print(mc_question)
    for option in mc_options:
        print(option)
    print(f"Correct Answer: {mc_answer}")
    
    print("\nTrue/False Question:")
    tf_statement, tf_answer = generate_question(model, scaler, 'true_false')
    print(tf_statement)
    print(f"Correct Answer: {tf_answer}")

/Users/rajesh/.pyenv/versions/3.11.4/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Test Accuracy: 0.0000

Multiple Choice Question:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Given the following climate conditions:
Global Temperature Anomaly: 0.17°C
Ocean Temperature: 0.19°C
CO2 Levels: 0.46 ppm
El Niño Index: 0.29

What is the most likely scenario for extreme weather events?

A) High probability of extreme weather (>0.20)
B) Moderate probability of extreme weather (-0.10 - 0.10)
C) Low probability of extreme weather (<-0.20)
D) No significant change in extreme weather patterns
Correct Answer: B

True/False Question:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Given the current climate conditions, the likelihood of extreme weather events is relatively low.
Correct Answer: False


/Users/rajesh/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/rajesh/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


## 7. Interactive Quiz Function


In [8]:
def take_quiz(model, scaler, num_questions=5):
    score = 0
    for i in range(num_questions):
        print(f"\nQuestion {i+1}:")
        if np.random.rand() > 0.5:
            # Multiple Choice Question
            question, options, correct_answer = generate_question(model, scaler, 'multiple_choice')
            print(question)
            for option in options:
                print(option)
            user_answer = input("Your answer (A/B/C/D): ").upper()
            if user_answer == correct_answer:
                print("Correct!")
                score += 1
            else:
                print(f"Incorrect. The correct answer is {correct_answer}.")
        else:
            # True/False Question
            statement, correct_answer = generate_question(model, scaler, 'true_false')
            print(statement)
            user_answer = input("True or False? (T/F): ").upper() == 'T'
            if user_answer == correct_answer:
                print("Correct!")
                score += 1
            else:
                print(f"Incorrect. The correct answer is {correct_answer}.")
    
    print(f"\nQuiz completed! Your score: {score}/{num_questions}")

## 8. Run the quiz


In [9]:
if __name__ == "__main__":
    print("\nWelcome to the Extreme Weather Events Quiz!")
    take_quiz(model, scaler)


Welcome to the Extreme Weather Events Quiz!

Question 1:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


/Users/rajesh/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Given the current climate conditions, the likelihood of extreme weather events is relatively low.
Incorrect. The correct answer is False.

Question 2:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


/Users/rajesh/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Given the following climate conditions:
Global Temperature Anomaly: 0.01°C
Ocean Temperature: 0.52°C
CO2 Levels: 0.04 ppm
El Niño Index: 0.57

What is the most likely scenario for extreme weather events?

A) High probability of extreme weather (>0.20)
B) Moderate probability of extreme weather (-0.10 - 0.10)
C) Low probability of extreme weather (<-0.20)
D) No significant change in extreme weather patterns
Incorrect. The correct answer is B.

Question 3:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step


...

Given the following climate conditions:
Global Temperature Anomaly: 0.97°C
Ocean Temperature: 0.54°C
CO2 Levels: 0.46 ppm
El Niño Index: 0.95

What is the most likely scenario for extreme weather events?

A) High probability of extreme weather (>0.20)
B) Moderate probability of extreme weather (-0.10 - 0.10)
C) Low probability of extreme weather (<-0.20)
D) No significant change in extreme weather patterns
Correct!

Question 4:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


/Users/rajesh/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Given the following climate conditions:
Global Temperature Anomaly: 0.06°C
Ocean Temperature: 0.47°C
CO2 Levels: 0.24 ppm
El Niño Index: 0.47

What is the most likely scenario for extreme weather events?

A) High probability of extreme weather (>0.20)
B) Moderate probability of extreme weather (-0.10 - 0.10)
C) Low probability of extreme weather (<-0.20)
D) No significant change in extreme weather patterns
Correct!

Question 5:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


/Users/rajesh/.pyenv/versions/3.11.4/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Given the current climate conditions, the likelihood of extreme weather events is relatively low.
Correct!

Quiz completed! Your score: 3/5
